In [1]:
from itertools import cycle
import numpy as np
from decimal import Decimal

In [2]:
def recon_command(origin_list):
    out = ""
    for el in origin_list:
        out+=el+" "
    return out[:-1]

In [3]:
def hyperparameterbegin(token):
    if "--" in token:
        return True
    return False

In [4]:
def gethyperparameter(token):
    return token[2:]

In [5]:
def gethyperdict(command_list):
    #command_list = command.split(" ")
    hyperdict={}
    is_hyper = False
    before_el = None
    for idx,el in enumerate(command_list):
        if is_hyper:
            hyper_name = gethyperparameter(before_el)
            hyperdict[hyper_name] = {}
            hyperdict[hyper_name]["val"] = el
            hyperdict[hyper_name]["val_idx"] = idx
        is_hyper = hyperparameterbegin(el)
        before_el = el   
    return hyperdict

In [6]:
class hyper_candidate():
    def __init__(self, name = None, val_candidate = None,period = 1,*args,**kwargs):
        self.name = name
        if type(val_candidate) == int:
            val_candidate = range(val_candidate)

        self.val_candidate = cycle(val_candidate)
        
        
        
        self.period = period
        self.state = cycle(range(self.period))
        self.return_val = None
    
    def get_name(self):
        return self.name
    
    def get_next_val(self):
        if next(self.state) == 0:
            self.return_val = next(self.val_candidate)
        return self.return_val

In [7]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [18]:
def change_experiment_name(exp_name, hyp_name, hyp_val):
    if hyp_name not in exp_name:
        recon_name = exp_name + hyp_name + str((hyp_val))
    else:
        point = exp_name.split(hyp_name)
        reside = point[0]
        cut = point[1]    
        count = 0
        while(isfloat(cut[:count+1])):
            count+=1
            if count == len(cut):
                break
        recon_name = reside + hyp_name + str((hyp_val))+cut[count:]
    return recon_name



In [19]:
def exist_experient_name(name,hyper_dict):
    try:
        hyper_dict[name]
        return hyper_dict[name]['val_idx']
    except ValueError:
        return -1

In [36]:
def change_hyper_val(command_list,hyperdict, list_inform_of_hyper_candidate,command_num):
    exp_name = 'additional_experiment_name'
    additional_experiment_name_idx = exist_experient_name(exp_name,hyperdict)
    
    list_of_hyper_candidate = []
    for inform in list_inform_of_hyper_candidate:
        list_of_hyper_candidate.append(hyper_candidate(*inform))
    result_command = {}
    for i in range(command_num):
        for hyp in list_of_hyper_candidate:
            hyp_name = hyp.get_name()
            hyp_val = hyp.get_next_val()
            command_list[hyperdict[hyp_name]['val_idx']] = str((hyp_val))
            if (additional_experiment_name_idx>=0) and (hyp_name!='cuda_visible_device'):
                command_list[additional_experiment_name_idx] = change_experiment_name(command_list[additional_experiment_name_idx], hyp_name, hyp_val)
        result_command[i] = recon_command(command_list)
    
    return result_command
    
    

In [37]:
def print_result_command(result_command):
    for i in range(len(result_command)):
        print("#################################################")
        print(i, "th command ")
        print("#################################################")
        print(result_command[i])
        print("*************************************************")

In [38]:
def make_sweep(command,list_inform_of_hyper_candidate, command_num):
    command_list = command.split(" ")
    hyperdict = gethyperdict(command_list)
    result = change_hyper_val(command_list,hyperdict, list_inform_of_hyper_candidate,command_num)
    print_result_command(result)
    return result

In [39]:
def xfrange(start, stop, step,maximum_float_precision = 2):
    i = 0
    while start + i * step < stop:
        yield '%.2f' % (start + i * step)
        i += 1


In [40]:
command = "python3 federated_train.py --method FedMOSAM --config configs/cifar_Fedavg.yaml --data ./data --mode dirichlet --dirichlet_alpha 0.3 --batch_size 50 --centralized_epochs 0 --global_epochs 1000 --local_epochs 5 --epsilon 0.1 --momentum 0 --lr 0.1 --learning_rate_decay 0.998 --weight_decay 1e-3 --seed 0 --set CIFAR100 --arch ResNet18 --workers 0 --cuda_visible_device 2 --alpha 0 --num_of_clients 100 --participation_rate 0.05 --additional_experiment_name ld0.998parti0.05_beta0.1_rho0.05 --global_epochs 10000 --global_method global_delta --beta 0.1 --rho 0.05"

#ex) "This is example with hyperparameter --a 1 --b 2"

In [44]:
list_inform_of_hyper_candidate = [("cuda_visible_device",range(4,5)),("rho",xfrange(0.4,0.5,0.05))]

#ex) [("a",5),("b",range(0,7,2))]

In [45]:
command_num = 10

#ex) 10

In [46]:
result_command = make_sweep(command,list_inform_of_hyper_candidate, command_num)

#################################################
0 th command 
#################################################
python3 federated_train.py --method FedMOSAM --config configs/cifar_Fedavg.yaml --data ./data --mode dirichlet --dirichlet_alpha 0.3 --batch_size 50 --centralized_epochs 0 --global_epochs 1000 --local_epochs 5 --epsilon 0.1 --momentum 0 --lr 0.1 --learning_rate_decay 0.998 --weight_decay 1e-3 --seed 0 --set CIFAR100 --arch ResNet18 --workers 0 --cuda_visible_device 4 --alpha 0 --num_of_clients 100 --participation_rate 0.05 --additional_experiment_name ld0.998parti0.05_beta0.1_rho0.40 --global_epochs 10000 --global_method global_delta --beta 0.1 --rho 0.40
*************************************************
#################################################
1 th command 
#################################################
python3 federated_train.py --method FedMOSAM --config configs/cifar_Fedavg.yaml --data ./data --mode dirichlet --dirichlet_alpha 0.3 --batch_size 50 --centraliz